# DSPy Concise COT

In [18]:
#|default_exp dspy.predict.ccot

In [19]:
#|hide
from fastcore.test import *
from nbdev.showdoc import *

In [20]:
#|export

import dspy
from dspy.primitives.program import Module
from dspy.signatures.signature import ensure_signature

from bellem.logging import get_logger

log = get_logger(__name__)

In [21]:

class ConciseChainOfThought(Module):
    def __init__(self, signature, rationale_type=None, **config):
        super().__init__()

        signature = ensure_signature(signature)

        prefix = "Reasoning: Let's concisely think step by step in order to"
        desc = "${reasoning}"
        rationale_type = rationale_type or dspy.OutputField(prefix=prefix, desc=desc)
        extended_signature = signature.prepend("reasoning", rationale_type, type_=str)
        
        self.predict = dspy.Predict(extended_signature, **config)

    def forward(self, **kwargs):
        return self.predict(**kwargs)


In [22]:
from dotenv import load_dotenv
load_dotenv()

True

In [23]:
import dspy
from bellem.dspy.utils import configure_lm

lm = configure_lm('openai/gpt-4o-mini')

In [24]:
context = """
Glenhis Hernández (born 7 October 1990 in Havana) is a taekwondo practitioner from Cuba. She was the 2013 World
Champion in middleweight.

The current mayor of Havana ("President of the People's Power Provincial Assembly") is Marta Hernández Romero, she
was elected on March 5, 2011.
""".strip()

question = 'Who is the current mayor of the city Glenhis Hernández was born?'

In [25]:
cot = dspy.ChainOfThought('context,question -> answer')
prediction = cot(context=context, question=question)
prediction

Prediction(
    reasoning='Glenhis Hernández was born in Havana, Cuba. According to the context provided, the current mayor of Havana is Marta Hernández Romero, who was elected on March 5, 2011. Therefore, the answer to the question about the current mayor of the city where Glenhis Hernández was born is Marta Hernández Romero.',
    answer='Marta Hernández Romero'
)

In [26]:
ccot = ConciseChainOfThought('context,question -> answer')
prediction = ccot(context=context, question=question)
prediction

Prediction(
    reasoning='Glenhis Hernández was born in Havana, Cuba. According to the context provided, the current mayor of Havana is Marta Hernández Romero, who was elected on March 5, 2011. Therefore, the answer to the question regarding the current mayor of the city where Glenhis Hernández was born is Marta Hernández Romero.',
    answer='Marta Hernández Romero'
)

In [27]:
def format_message(message):
    return f"{message['role']}: {message['content']}"

In [28]:
for msg in lm.history[0]['messages']:
    print(format_message(msg))  
    print()

system: Your input fields are:
1. `context` (str)
2. `question` (str)

Your output fields are:
1. `reasoning` (str)
2. `answer` (str)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## context ## ]]
{context}

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `context`, `question`, produce the fields `answer`.

user: [[ ## context ## ]]
Glenhis Hernández (born 7 October 1990 in Havana) is a taekwondo practitioner from Cuba. She was the 2013 World
Champion in middleweight.

The current mayor of Havana ("President of the People's Power Provincial Assembly") is Marta Hernández Romero, she
was elected on March 5, 2011.

[[ ## question ## ]]
Who is the current mayor of the city Glenhis Hernández was born?

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ##

In [29]:
for msg in lm.history[1]['messages']:
    print(format_message(msg))  
    print()

system: Your input fields are:
1. `context` (str)
2. `question` (str)

Your output fields are:
1. `reasoning` (str)
2. `answer` (str)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## context ## ]]
{context}

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `context`, `question`, produce the fields `answer`.

user: [[ ## context ## ]]
Glenhis Hernández (born 7 October 1990 in Havana) is a taekwondo practitioner from Cuba. She was the 2013 World
Champion in middleweight.

The current mayor of Havana ("President of the People's Power Provincial Assembly") is Marta Hernández Romero, she
was elected on March 5, 2011.

[[ ## question ## ]]
Who is the current mayor of the city Glenhis Hernández was born?

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ##

In [30]:
#|hide
import nbdev; nbdev.nbdev_export()